# eof - Multi-Sensor Demo

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/profLewis/eof/blob/main/notebooks/demo_multisensor.ipynb)

This notebook demonstrates multi-sensor data retrieval with `eof`:

1. Fetch Sentinel-2 and Landsat data
2. Inspect the `EOResult` dataclass and footprint ID maps
3. Compare NDVI time series across sensors
4. Visualise footprint maps
5. Fetch multiple sensors in parallel
6. Fetch MODIS data

**No credentials needed** — uses AWS (Sentinel-2, free) and Planetary Computer (Landsat, MODIS, free).

For **GEE-only sensors** (VIIRS, Sentinel-3 OLCI), run `earthengine authenticate` first.

## 1. Install eof

In [ ]:
!pip install -q https://github.com/profLewis/eof/archive/refs/heads/main.zip

## 2. Check supported sensors and platforms

In [ ]:
import eof

print('Supported sensors:', eof.SENSORS)
print()
for sensor, platforms in eof.SENSOR_PLATFORMS.items():
    print(f'  {sensor:12s} -> {platforms}')
print()
print('Available sources:', eof.get_available_sources())
print(f'Test GeoJSON: {eof.TEST_GEOJSON}')

## 3. Fetch Sentinel-2 data

In [ ]:
s2 = eof.get_eo_data(
    'sentinel2',
    start_date='2022-07-15',
    end_date='2022-11-30',
    geojson_path=eof.TEST_GEOJSON,
    source='aws',
)

print(f'Sensor:            {s2.sensor}')
print(f'Band names:        {s2.band_names}')
print(f'Reflectance shape: {s2.reflectance.shape}  (images, bands, H, W)')
print(f'DOYs:              {s2.doys}')
print(f'Native resolutions: {s2.native_resolutions}')
print(f'Footprint maps:    {list(s2.footprints.keys())} m')
for res, fp in s2.footprints.items():
    print(f'  {res}m footprints: shape={fp.shape}, dtype={fp.dtype}, '
          f'n_unique={len(set(fp.ravel()))}') 

## 4. Fetch Landsat data

In [ ]:
landsat = eof.get_eo_data(
    'landsat',
    start_date='2022-07-15',
    end_date='2022-11-30',
    geojson_path=eof.TEST_GEOJSON,
    source='planetary',  # Landsat on AWS is requester-pays; Planetary is free
)

print(f'Sensor:            {landsat.sensor}')
print(f'Band names:        {landsat.band_names}')
print(f'Reflectance shape: {landsat.reflectance.shape}')
print(f'DOYs:              {landsat.doys}')
print(f'Native resolutions: {landsat.native_resolutions}')
print(f'Footprint maps:    {list(landsat.footprints.keys())} m')
fp30 = landsat.footprints[30]
print(f'  30m footprints: shape={fp30.shape}, dtype={fp30.dtype}, '
      f'n_unique={len(set(fp30.ravel()))}')

## 5. Compare NDVI time series

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_mean_ndvi(result, nir_idx, red_idx):
    """Compute spatially-averaged NDVI time series."""
    nir = result.reflectance[:, nir_idx]
    red = result.reflectance[:, red_idx]
    ndvi = (nir - red) / (nir + red)
    mean_ndvi = np.nanmean(ndvi, axis=(1, 2))
    return mean_ndvi

# S2: NIR = B08 (index 6), Red = B04 (index 2)
s2_ndvi = compute_mean_ndvi(s2, nir_idx=6, red_idx=2)

# Landsat: NIR = SR_B5 (index 4), Red = SR_B4 (index 3)
ls_ndvi = compute_mean_ndvi(landsat, nir_idx=4, red_idx=3)

plt.figure(figsize=(12, 5))

valid_s2 = np.isfinite(s2_ndvi)
plt.plot(s2.doys[valid_s2], s2_ndvi[valid_s2], 'o--', color='#2196F3',
         label=f'Sentinel-2 ({s2.reflectance.shape[0]} images)', markersize=6)

valid_ls = np.isfinite(ls_ndvi)
plt.plot(landsat.doys[valid_ls], ls_ndvi[valid_ls], 's--', color='#FF9800',
         label=f'Landsat ({landsat.reflectance.shape[0]} images)', markersize=8)

plt.xlabel('Day of Year (2022)')
plt.ylabel('NDVI')
plt.title('Multi-Sensor NDVI Time Series')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Visualise footprint ID maps

Footprint maps show which coarse-resolution pixel each 10m pixel belongs to.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# S2 10m footprints
im0 = axes[0].imshow(s2.footprints[10], cmap='tab20')
axes[0].set_title(f'S2 10m footprints\n({len(np.unique(s2.footprints[10]))} unique)')
axes[0].axis('off')

# S2 20m footprints
im1 = axes[1].imshow(s2.footprints[20], cmap='tab20')
axes[1].set_title(f'S2 20m footprints\n({len(np.unique(s2.footprints[20]))} unique)')
axes[1].axis('off')

# Landsat 30m footprints
im2 = axes[2].imshow(landsat.footprints[30], cmap='tab20')
axes[2].set_title(f'Landsat 30m footprints\n({len(np.unique(landsat.footprints[30]))} unique)')
axes[2].axis('off')

plt.suptitle('Footprint ID Maps (10m grid)', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Side-by-side NDVI maps

In [ ]:
# Pick a date near DOY 280 for both sensors
s2_idx = np.argmin(np.abs(s2.doys - 280))
ls_idx = np.argmin(np.abs(landsat.doys - 280))

s2_nir = s2.reflectance[s2_idx, 6]
s2_red = s2.reflectance[s2_idx, 2]
s2_ndvi_map = (s2_nir - s2_red) / (s2_nir + s2_red)

ls_nir = landsat.reflectance[ls_idx, 4]
ls_red = landsat.reflectance[ls_idx, 3]
ls_ndvi_map = (ls_nir - ls_red) / (ls_nir + ls_red)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im0 = axes[0].imshow(s2_ndvi_map, vmin=0, vmax=1, cmap='RdYlGn')
axes[0].set_title(f'Sentinel-2 NDVI\nDOY {s2.doys[s2_idx]}')
axes[0].axis('off')

im1 = axes[1].imshow(ls_ndvi_map, vmin=0, vmax=1, cmap='RdYlGn')
axes[1].set_title(f'Landsat NDVI\nDOY {landsat.doys[ls_idx]}')
axes[1].axis('off')

fig.colorbar(im0, ax=axes, shrink=0.7, label='NDVI')
plt.suptitle('NDVI Comparison (nearest date to DOY 280)', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Fetch multiple sensors in parallel

In [ ]:
results = eof.get_multi_sensor_data(
    ['sentinel2', 'landsat'],
    start_date='2022-07-15',
    end_date='2022-11-30',
    geojson_path=eof.TEST_GEOJSON,
    # source='auto' picks the best free platform per sensor
)

for sensor_name, result in results.items():
    print(f'{sensor_name}:')
    print(f'  shape:      {result.reflectance.shape}')
    print(f'  bands:      {result.band_names}')
    print(f'  N images:   {result.reflectance.shape[0]}')
    print(f'  footprints: {list(result.footprints.keys())} m')

## 9. MODIS (Planetary Computer, no credentials needed)

MODIS on Planetary Computer is 8-day composite (MOD09A1). For daily MODIS/VIIRS, use NASA Earthdata or GEE.

In [ ]:
modis = eof.get_eo_data(
    'modis',
    start_date='2022-07-15',
    end_date='2022-11-30',
    geojson_path=eof.TEST_GEOJSON,
    source='planetary',
)
print(f'MODIS: {modis.reflectance.shape}, bands={modis.band_names}')
print(f'DOYs: {modis.doys}')
print(f'Footprints: {list(modis.footprints.keys())} m')
for res, fp in modis.footprints.items():
    print(f'  {res}m: dtype={fp.dtype}, n_unique={len(np.unique(fp))}')

## 10. Spectral Response Functions (Bandpass)

Each `EOResult` includes the sensor's spectral response functions.

In [ ]:
# Bandpass data from result
bp = s2.bandpass
print(f'Wavelength range: {bp["wavelength_nm"][0]:.0f} - {bp["wavelength_nm"][-1]:.0f} nm')
print(f'Response shape:   {bp["response"].shape}  (bands x wavelength)')
print(f'Band names:       {bp["band_names"]}')
print(f'Center wavelengths: {bp["center_wavelength_nm"]}')
print(f'FWHM:             {bp["fwhm_nm"]}')

# Plot SRFs for all three sensors
fig, axes = plt.subplots(1, 3, figsize=(18, 4))
for ax, (name, result) in zip(axes, [('Sentinel-2', s2), ('Landsat', landsat), ('MODIS', modis)]):
    bp = result.bandpass
    for i, band in enumerate(bp['band_names']):
        ax.plot(bp['wavelength_nm'], bp['response'][i], label=band)
    ax.set_xlabel('Wavelength (nm)')
    ax.set_ylabel('Response')
    ax.set_title(f'{name} Spectral Response Functions')
    ax.set_xlim(350, 2600)
    ax.legend(fontsize=7, ncol=2)
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. VIIRS, Sentinel-3 OLCI (requires GEE)

These sensors are only available via Google Earth Engine.
Run `earthengine authenticate` first, or skip this section.

In [ ]:
# Uncomment to fetch VIIRS data via GEE
# viirs = eof.get_eo_data('viirs', '2022-07-15', '2022-11-30',
#                          eof.TEST_GEOJSON, source='gee')
# print(f'VIIRS: {viirs.reflectance.shape}, bands={viirs.band_names}')

# Uncomment to fetch Sentinel-3 OLCI data via GEE
# s3 = eof.get_eo_data('s3olci', '2022-07-15', '2022-11-30',
#                       eof.TEST_GEOJSON, source='gee')
# print(f'S3 OLCI: {s3.reflectance.shape}, bands={s3.band_names}')

print('Skipped (requires GEE credentials)')

## 12. Validation

In [ ]:
def validate_result(result, sensor_name):
    print(f'\n--- {sensor_name} ---')
    print(f'  reflectance dtype:  {result.reflectance.dtype}')
    print(f'  reflectance range:  [{np.nanmin(result.reflectance):.4f}, '
          f'{np.nanmax(result.reflectance):.4f}]')
    print(f'  N bands:            {result.reflectance.shape[1]}')
    print(f'  N dates:            {result.reflectance.shape[0]}')
    print(f'  mask dtype:         {result.mask.dtype}')
    print(f'  sensor field:       {result.sensor}')
    print(f'  band_names:         {result.band_names}')
    print(f'  footprint keys:     {list(result.footprints.keys())}')
    print(f'  bandpass:           {result.bandpass is not None}')
    
    assert result.reflectance.dtype == np.float32
    assert result.mask.dtype == bool
    assert result.sensor == sensor_name
    assert len(result.band_names) == result.reflectance.shape[1]
    assert len(result.footprints) > 0
    assert result.bandpass is not None
    for res, fp in result.footprints.items():
        assert fp.shape == result.reflectance.shape[2:]
    print(f'  PASSED')

validate_result(s2, 'sentinel2')
validate_result(landsat, 'landsat')
validate_result(modis, 'modis')

print('\nAll checks passed!')